In [1]:
import os
import sys
import argparse
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/CREDIT/libs/')
import verif_utils as vu

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## Functions

In [4]:
def sp_avg(DS, wlat):
    return DS.weighted(wlat).mean(['lat', 'lon'], skipna=True)

## Verificaition setup

The following inputs are needed:

(1) range of indices

(2) required lead times to verify

(3) save location

Note: lead time has to be 6 hourly or longer

In [12]:
verif_ind_start = 200; verif_ind_end = 205

leads_do = np.arange(6, 240+6, 6)
leads_do = leads_do - 1 # -1 for Python indexing

path_verif = '/glade/derecho/scratch/ksha/CREDIT/verif/wxformer_acc_{}_{}.nc'.format(verif_ind_start, verif_ind_end)

# ERA5 and fcst climatology info
ERA5_path_string = '/glade/campaign/cisl/aiml/ksha/CREDIT/gathered/ERA5_climo/ERA5_DOY{:05d}.nc'
OURS_clim_string = '/glade/campaign/cisl/aiml/gathered/forecast_climo/medium_boy_DOY{:03d}_LEAD{:03d}.nc'


In [13]:
# Adjust your file paths and chunk sizes as necessary
filename_ERA5 = sorted(glob('/glade/campaign/cisl/aiml/wchapman/MLWPS/STAGING/TOTAL_*'))
filename_ERA5 = filename_ERA5[-6:-3]
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')
    
# Select the specified variables and their levels
variables_levels = {
    'V500': None, 
    'U500': None, 
    'T500': None, 
    'Q500': None, 
    'Z500': None,
    'SP': None, 
    't2m': None,}

ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)
ds_ERA5_merge = ds_ERA5_merge.rename({'latitude':'lat','longitude':'lon'})

# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob('/glade/campaign/cisl/aiml/gathered/*.nc'))
filename_OURS_backup = sorted(glob('/glade/campaign/cisl/aiml/ksha/CREDIT/gathered/*.nc'))

# manual input bad files in '/glade/campaign/cisl/aiml/gathered/'
# provide replacements in '/glade/campaign/cisl/aiml/ksha/CREDIT/gathered/'
# correct file info and rerun climo days/leads that touchs the bad files
ind_bad = [206, 209, 211, 215, 360, 390, 400]
filename_bad = []

for i, i_bad in enumerate(ind_bad):
    file_old = filename_OURS[i_bad]
    file_new = filename_OURS_backup[i]

    if os.path.basename(file_old) == os.path.basename(file_new):
        filename_bad.append(file_new)
        filename_OURS[i_bad] = filename_OURS_backup[i]
    else:
        print('Replacement of bad file {} not found'.format(file_old))
        raise

filename_OURS = [fn for fn in filename_OURS if '2018' in fn or '2019' in fn or '2020' in fn]

L_max = len(filename_OURS)
assert verif_ind_end <= L_max, 'verified indices (days) exceeds the max index available'

filename_OURS = filename_OURS[verif_ind_start:verif_ind_end]

# latitude weighting
lat = xr.open_dataset(filename_OURS[0])["lat"]
w_lat = np.cos(np.deg2rad(lat))
w_lat = w_lat / w_lat.mean()

In [14]:
acc_results = []

for fn_ours in filename_OURS:
    # --------------------------------------------------------------- #
    # import and subset forecast
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variables_levels)
    ds_ours = ds_ours.isel(time=leads_do)
    
    # --------------------------------------------------------------- #
    # get ERA5 verification target
    ds_target = ds_ERA5_merge.sel(time=ds_ours['time']).compute()

    # --------------------------------------------------------------- #
    # get ERA5 climatology
    # pull day of year for anomaly computation
    dayofyear_values = ds_target['time.dayofyear'].values
    hourofday_values = ds_target['time'].dt.hour
    
    required_ERA5_clim = [
        ERA5_path_string.format(day) for day in dayofyear_values]
    
    for fn_required in required_ERA5_clim:
        if os.path.exists(fn_required) is False:
            print('Missing: {}'.format(fn_required))
            raise
        
    print('ERA5 climatology file requirments fulfilled')
    
    # open all ERA5 climatology files and merge as one
    ds_ERA5_clim = [xr.open_dataset(fn) for fn in required_ERA5_clim]
    ds_clim_merge = xr.concat(ds_ERA5_clim, dim='time')
    
    # unify coord names
    ds_clim_merge = ds_clim_merge.rename({'latitude':'lat','longitude':'lon'})
    ds_clim_merge['time'] = ds_target['time']
    
    # ========================================== #
    # ERA5 anomaly
    ds_anomaly_ERA5 = ds_target - ds_clim_merge
    # ========================================== #
    
    # --------------------------------------------------------------- #
    # get forecast climatology
    required_OURS_clim = [OURS_clim_string.format(day, (leads_do[i_day])+1) for i_day, day in enumerate(ds_ours['time.dayofyear'])]
    
    for fn_required in required_OURS_clim:
        if os.path.exists(fn_required) is False:
            print('Missing: {}'.format(fn_required))
            raise
        
    print('OURS climatology file requirments fulfilled')
    
    # open all fcst climatology files and merge as one
    datasets_f = [xr.open_dataset(fn) for fn in required_OURS_clim]

    # unify coord names
    fcst_clim = xr.concat(datasets_f, dim='time')
    fcst_clim = fcst_clim.drop_vars('level')
    fcst_clim['time'] = ds_ours['time']

    # ========================================== #
    # fcst anomaly
    ds_anomaly_OURS = ds_ours - fcst_clim
    
    # ========================================== #
    # anmalies --> ACC with latitude-based cosine weights (check sp_avg and w_lat)
    top = sp_avg(ds_anomaly_OURS*ds_anomaly_ERA5, w_lat)
    
    bottom = np.sqrt(
        sp_avg(ds_anomaly_OURS**2, w_lat) * sp_avg(ds_anomaly_ERA5**2, w_lat))
                
    acc_results.append((top/bottom).drop_vars('time'))
    
    print('ACC completed: {}'.format(fn_ours))
    
# Combine ACC results
ds_acc = xr.concat(acc_results, dim='days')

# Save
print('Save to {}'.format(path_verif))
ds_acc.to_netcdf(path_verif)

ERA5 climatology file requirments fulfilled
OURS climatology file requirments fulfilled
ACC completed: /glade/campaign/cisl/aiml/gathered/2018-04-11T00Z.nc
ERA5 climatology file requirments fulfilled
OURS climatology file requirments fulfilled
ACC completed: /glade/campaign/cisl/aiml/gathered/2018-04-11T12Z.nc
ERA5 climatology file requirments fulfilled
OURS climatology file requirments fulfilled
ACC completed: /glade/campaign/cisl/aiml/gathered/2018-04-12T00Z.nc
ERA5 climatology file requirments fulfilled
OURS climatology file requirments fulfilled
ACC completed: /glade/campaign/cisl/aiml/gathered/2018-04-12T12Z.nc
ERA5 climatology file requirments fulfilled
OURS climatology file requirments fulfilled
ACC completed: /glade/campaign/cisl/aiml/gathered/2018-04-13T00Z.nc
Save to /glade/derecho/scratch/ksha/CREDIT/verif/wxformer_acc_200_205.nc


In [15]:
ds_target['time'].dt.hour

<xarray.DataArray 'hour' (time: 40)> Size: 320B
array([ 6, 12, 18,  0,  6, 12, 18,  0,  6, 12, 18,  0,  6, 12, 18,  0,  6, 12,
       18,  0,  6, 12, 18,  0,  6, 12, 18,  0,  6, 12, 18,  0,  6, 12, 18,  0,
        6, 12, 18,  0])
Coordinates:
  * time     (time) datetime64[ns] 320B 2018-04-13T06:00:00 ... 2018-04-23

In [8]:
ds_ours

<xarray.Dataset> Size: 252MB
Dimensions:  (time: 11, lat: 640, lon: 1280)
Coordinates:
  * time     (time) datetime64[ns] 88B 2018-04-13T06:00:00 ... 2018-04-23
  * lat      (lat) float64 5kB 89.78 89.51 89.23 88.95 ... -89.23 -89.51 -89.78
  * lon      (lon) float64 10kB 0.0 0.2812 0.5625 0.8438 ... 359.2 359.4 359.7
Data variables:
    V500     (time, lat, lon) float32 36MB ...
    U500     (time, lat, lon) float32 36MB ...
    T500     (time, lat, lon) float32 36MB ...
    Q500     (time, lat, lon) float32 36MB ...
    Z500     (time, lat, lon) float32 36MB ...
    SP       (time, lat, lon) float32 36MB ...
    t2m      (time, lat, lon) float32 36MB ...

In [9]:
dayofyear_values

array([103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113])